- 소리 증폭

In [ ]:
import soundfile as sf
import numpy as np
import os

# 파일 경로 설정 
input_files = ['/content/정상.wav', '/content/비정상.wav', '/content/정상+비정상.wav']
output_files = ['/content/정상_증폭.wav', '/content/비정상_증폭.wav', '/content/정상+비정상_증폭.wav']

# 증폭 비율 설정 (2.0 = 2배 증폭)
amplification_factor = 2.0

# 파일 증폭 함수
def amplify_audio(input_path, output_path, factor):
    # 오디오 파일 읽기
    data, samplerate = sf.read(input_path)
    
    # 증폭 적용
    amplified_data = data * factor
    
    # 클리핑 방지 (값을 -1.0 ~ 1.0으로 제한)
    amplified_data = np.clip(amplified_data, -1.0, 1.0)
    
    # 증폭된 파일 저장
    sf.write(output_path, amplified_data, samplerate)
    print(f'{output_path} 파일이 증폭되어 저장되었습니다.')

# 모든 파일 증폭
for input_file, output_file in zip(input_files, output_files):
    if os.path.exists(input_file):
        amplify_audio(input_file, output_file, amplification_factor)
    else:
        print(f"{input_file} 파일이 존재하지 않습니다.")

In [2]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
## 테스트용

import os
from pydub import AudioSegment

# 파일 경로 설정 (원하는 파일 하나씩 직접 지정)
normal_path = "ㅈㅅ/ㅈㅅ/1-7456-A-13.wav"
abnormal_path = "ㅂㅈㅅ/ㅂㅈㅅ/1.wav"
output_path = "mixed_data/test1.wav"

# 출력 폴더 생성
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# 오디오 불러오기
normal = AudioSegment.from_wav(normal_path)
abnormal = AudioSegment.from_wav(abnormal_path)

# 길이 맞추기
min_len = min(len(normal), len(abnormal))
normal = normal[:min_len]
abnormal = abnormal[:min_len]

# 오버레이 (동시에 재생)
mixed = normal.overlay(abnormal)

# 저장
mixed.export(output_path, format="wav")
print(f"Saved: {output_path}")

Saved: mixed_data/test1.wav


In [ ]:
import os
import random
from glob import glob
from pydub import AudioSegment

# 입력/출력 디렉토리
BASE_DIR = "data"
OUTPUT_DIR = "mixed_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 조합 규칙 정의
pairings = [
    {
        "normal": ("일상", "물 내리는 소리"),
        "abnormal": ("강도",),
        "label": "water_scream"
    },
    # 추가 매핑 필요 시 여기에 append
]

def get_wav_files(base, *subdirs):
    path = os.path.join(BASE_DIR, base, *subdirs)
    return glob(os.path.join(path, "*.wav"))

# 조합 및 저장
for pair in pairings:
    normal_files = get_wav_files("정상", *pair["normal"])
    abnormal_files = get_wav_files("비정상", *pair["abnormal"])
    
    # 짧은 쪽 기준으로 조합 수 제한
    count = min(len(normal_files), len(abnormal_files))

    for i in range(count):
        normal = AudioSegment.from_wav(normal_files[i])
        abnormal = AudioSegment.from_wav(abnormal_files[i])
        
        # 길이 맞추기
        min_len = min(len(normal), len(abnormal))
        normal = normal[:min_len]
        abnormal = abnormal[:min_len]

        # overlay 후 저장
        mixed = normal.overlay(abnormal)
        output_path = os.path.join(OUTPUT_DIR, f"{pair['label']}_{i+1}.wav")
        mixed.export(output_path, format="wav")

        print(f"Saved: {output_path}")